# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd

In [5]:
housing=pd.read_csv('cal_housing_clean.csv')

In [6]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [68]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [11]:
y_val=housing['medianHouseValue']

In [17]:
x_data=housing.drop('medianHouseValue',axis=1)

In [18]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [21]:
from sklearn.preprocessing import MinMaxScaler

In [22]:
scaler=MinMaxScaler()

In [23]:
#scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [24]:
x_train=pd.DataFrame(data=scaler.fit_transform(x_train),columns=x_train.columns,index=x_train.index)

In [25]:
x_test=pd.DataFrame(data=scaler.fit_transform(x_test),columns=x_test.columns,index=x_test.index)

In [26]:
scaler.transform(x_train)

array([[-0.01268743, -0.00020991, -0.00038037, -0.00022289, -0.00039533,
        -0.02464736],
       [-0.00768935, -0.00021198, -0.00040087, -0.00022402, -0.00041658,
        -0.03137012],
       [-0.00653595, -0.00021148, -0.00039771, -0.00022367, -0.0004132 ,
        -0.01979517],
       ...,
       [-0.01768551, -0.0002092 , -0.00038655, -0.0002229 , -0.00040265,
        -0.00762048],
       [-0.00615148, -0.00021163, -0.00039746, -0.00022372, -0.00041231,
        -0.02124674],
       [-0.01114956, -0.00020935, -0.00038134, -0.00022243, -0.00039713,
        -0.02414511]])

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [27]:
housing.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [28]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [32]:
age=tf.feature_column.numeric_column('housingMedianAge')
rooms=tf.feature_column.numeric_column('totalRooms')
bed_rooms=tf.feature_column.numeric_column('totalBedrooms')
population=tf.feature_column.numeric_column('population')
house_holds=tf.feature_column.numeric_column('households')
median_in_come=tf.feature_column.numeric_column('medianIncome')
median_house_value=tf.feature_column.numeric_column('medianHouseValue')


In [33]:
feat_cols=[age,rooms,bed_rooms,population,house_holds,median_in_come]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [36]:
input_function=tf.estimator.inputs.pandas_input_fn(x_train,y_train,batch_size=10,num_epochs=1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [37]:
model=tf.estimator.DNNRegressor(hidden_units=[6,6,6],feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\STEALE~1\\AppData\\Local\\Temp\\tmpihpsid_4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000272FF9D52B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

In [38]:
model.train(input_fn=input_function,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\STEALE~1\AppData\Local\Temp\tmpihpsid_4\model.ckpt.
INFO:tensorflow:loss = 930838600000.0, step = 1
INFO:tensorflow:global_step/sec: 521.118
INFO:tensorflow:loss = 599118200000.0, step = 101 (0.194 sec)
INFO:tensorflow:global_step/sec: 690.082
INFO:tensorflow:loss = 398409270000.0, step = 201 (0.145 sec)
INFO:tensorflow:global_step/sec: 685.328
INFO:tensorflow:loss = 592405100000.0, step = 301 (0.146 sec)
INFO:tensorflow:global_step/sec: 633.273
INFO:tensorflow:loss = 321981580000.0, step = 401 (0.157 sec)
INFO:tensorflow:global_step/sec: 552.799
INFO:tensorflow:loss = 441311040000.0, step = 501 (0.183 sec)
INFO:tensorflow:global_step/sec: 719.84
INFO:tensorflow:loss = 255302300000.0, step = 601 (0.

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [59]:
predict_fun=tf.estimator.inputs.pandas_input_fn(x_test,num_epochs=1,shuffle=False)

In [60]:
pre=model.predict(input_fn=predict_fun)

In [61]:
predictions=list(pre)
predictions

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\STEALE~1\AppData\Local\Temp\tmpihpsid_4\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([146193.88], dtype=float32)},
 {'predictions': array([167424.08], dtype=float32)},
 {'predictions': array([145129.62], dtype=float32)},
 {'predictions': array([116647.79], dtype=float32)},
 {'predictions': array([155760.5], dtype=float32)},
 {'predictions': array([136882.36], dtype=float32)},
 {'predictions': array([152618.2], dtype=float32)},
 {'predictions': array([135659.67], dtype=float32)},
 {'predictions': array([133945.], dtype=float32)},
 {'predictions': array([102858.195], dtype=float32)},
 {'predictions': array([137725.89], dtype=float32)},
 {'predictions': array([156559.98], dtype=float32)},
 {'predictions': array([127009.945], dtype=float32)},
 {'predictions': array([114799.79], dtype=float32)},
 {'predictions': array([168644.31], dtype=float32)},
 {'predictions': array([109579.98], dtype=float32)},
 {'predictions': array([133344.67], dtype=float32)},
 {'predictions': array([120221.98], dtype=float32)},
 {'predictions': array([119166.07], dtype=float3

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [62]:
final_pre=[]
for pre in predictions:
    final_pre.append(pre['predictions'])

In [63]:
final_pre

[array([146193.88], dtype=float32),
 array([167424.08], dtype=float32),
 array([145129.62], dtype=float32),
 array([116647.79], dtype=float32),
 array([155760.5], dtype=float32),
 array([136882.36], dtype=float32),
 array([152618.2], dtype=float32),
 array([135659.67], dtype=float32),
 array([133945.], dtype=float32),
 array([102858.195], dtype=float32),
 array([137725.89], dtype=float32),
 array([156559.98], dtype=float32),
 array([127009.945], dtype=float32),
 array([114799.79], dtype=float32),
 array([168644.31], dtype=float32),
 array([109579.98], dtype=float32),
 array([133344.67], dtype=float32),
 array([120221.98], dtype=float32),
 array([119166.07], dtype=float32),
 array([140418.19], dtype=float32),
 array([113472.49], dtype=float32),
 array([157249.94], dtype=float32),
 array([143721.08], dtype=float32),
 array([109906.7], dtype=float32),
 array([130449.414], dtype=float32),
 array([158263.33], dtype=float32),
 array([127704.49], dtype=float32),
 array([99945.64], dtype=float

In [64]:
from sklearn.metrics import mean_squared_error

In [65]:
mean_squared_error(y_true=y_test,y_pred=final_pre)**0.5

129726.17483471935

# Great Job!